# Anti-Money Laundering Pattern Analysis

This notebook demonstrates various money laundering patterns and how they can be detected using the DeepAML system.

In [1]:
import sys
sys.path.append('../')

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from src.data_generator import TransactionGenerator
from src.visualizations import TransactionVisualizer
from src.feature_extractor import FeatureExtractor
from src.risk_scorer import RiskScorer

/Users/sahillihas/Documents/Self_Projects/DeepAML/.deepaml_venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Generate Sample Data

First, let's generate some sample transaction data with known patterns:

In [2]:
# Initialize generator
generator = TransactionGenerator(num_accounts=1000)

# Generate dataset
transactions = generator.generate_dataset(num_days=90, suspicious_ratio=0.15)

# Convert to DataFrame
df = pd.DataFrame(transactions)
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(f"Generated {len(transactions)} transactions")
print(f"Time range: {df['timestamp'].min()} to {df['timestamp'].max()}")

ValueError: unconverted data remains when parsing with format "%Y-%m-%dT%H:%M:%S": ".878797", at position 501. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

## 1. Structuring (Smurfing) Pattern

Structuring involves breaking down a large transaction into smaller ones to avoid reporting thresholds.

In [3]:
visualizer = TransactionVisualizer(dark_mode=True)

# Get structuring pattern data
pattern_data = {'transactions': transactions[:10]}  # Using first 10 transactions for example
fig = visualizer._visualize_structuring(pattern_data['transactions'])
fig.show()

## 2. Layering Pattern

Layering involves moving money through multiple accounts to obscure its origin.

In [4]:
# Visualize layering pattern
fig = visualizer._visualize_layering(pattern_data['transactions'])
fig.show()

## 3. Round-Tripping Pattern

Round-tripping involves moving money in a circle through multiple accounts.

In [5]:
# Visualize round-tripping pattern
fig = visualizer._visualize_round_tripping(pattern_data['transactions'])
fig.show()

## 4. Transaction Network Analysis

Let's analyze the complete transaction network:

In [6]:
# Create and display network graph
G = visualizer.create_transaction_network(transactions, 'network_analysis.html')

# Basic network statistics
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

FileNotFoundError: [Errno 2] No such file or directory: ''

## 5. Risk Analysis

Let's analyze the risk distribution across transactions:

In [8]:
from tqdm import tqdm
# Extract features and calculate risk scores
feature_extractor = FeatureExtractor()
risk_scorer = RiskScorer()

risk_scores = []
for tx in tqdm(transactions):
    features = feature_extractor.extract_features(tx, transactions)
    risk_score = risk_scorer.calculate_risk_score(features)
    risk_scores.append(risk_score)

# Plot risk score distribution
fig = px.histogram(risk_scores, title='Risk Score Distribution')
fig.show()

100%|██████████| 10724/10724 [00:49<00:00, 218.31it/s]


## 6. Country Risk Analysis

Analyze transaction patterns by country:

In [9]:
# Group by country and sum transaction amounts
country_amounts = df.groupby('sender_country')['amount'].agg(['sum', 'count']).reset_index()

# Create bubble chart
fig = px.scatter(country_amounts, 
                x='sum', 
                y='count',
                size='sum',
                hover_name='sender_country',
                title='Country Transaction Analysis')
fig.show()